In [1]:
import torch
from torch import optim
from torch.autograd import Variable
from torch.utils.data import DataLoader

In [2]:
from importlib import reload
# import [module]
# importlib.reload([module])

In [3]:
import numpy as np
import matplotlib.pyplot as plt
def show(x):
    plt.imshow(x)
    plt.show()

In [40]:
import custom.models
reload(custom.models)
from custom.models import seg_net

In [4]:
from custom.models import seg_net
from custom.utils import get_weight_c
from custom.datasets import nuclei_dataset, nuclei_data
from custom.loss import weighted_bce_loss_with_logits as loss_func

/root/.pyenv/versions/anaconda3-4.4.0/lib/python3.6/site-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.16) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [5]:
import time
ts = time.time()
dat = nuclei_data(path='../data/stage1_train/')
te = time.time()
print(te - ts)

0.0482785701751709


In [6]:
train_loader = DataLoader(nuclei_dataset(dat),batch_size=16, shuffle=True)
test_loader = DataLoader(nuclei_dataset(dat, train=False), batch_size=16)

In [7]:
# del model
torch.cuda.empty_cache()

In [41]:
# model = u_net(bn=True)
# model = u_net_res()
# model = u_net(bn=False)
# model.load_state_dict(torch.load('./models/unbn_1'))
model = seg_net()
model.cuda()

seg_net(
  (conv1_3): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv1_5): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv1_7): Conv2d(1, 64, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
  (norm1): BatchNorm2d(129, eps=1e-05, momentum=0.1, affine=True)
)

In [42]:
optimizer = optim.Adam(model.parameters(), lr=1e-3)
# optimizer = optim.SGD(model.parameters(), lr=1e-4, momentum=0.9)

In [23]:
from torchvision.utils import save_image
to_cuda_var = lambda x: Variable(x.float()).cuda()
def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, (im, mask, dist) in enumerate(train_loader):
        weight = get_weight_c(mask, factor01=(5, .5)) + 10. * np.exp(-np.power(dist.numpy(), 2) / 400.)
        weight = torch.from_numpy(weight)
        input, target, weight = to_cuda_var(im), to_cuda_var(mask), to_cuda_var(weight)
        
        optimizer.zero_grad()
        output = model(input)
        loss = loss_func(output, target, weight)
        loss.backward()
        train_loss += loss.data[0]
        optimizer.step()
        
        if batch_idx == 0:
            n = 8
            comparison = torch.cat([input[:n], output[:n]])
            save_image(comparison.data.cpu(),
                     './results/train_' + str(epoch) + '.png', nrow=n)

    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(train_loader.dataset)))

In [24]:
from torchvision.utils import save_image
to_cuda_var = lambda x: Variable(x.float()).cuda()
from torch.nn.functional import threshold
def test(epoch):
    model.eval()
    test_loss = 0
    for i, (im, mask, dist) in enumerate(test_loader):
        weight = get_weight_c(mask, factor01=(5, .5)) + 10. * np.exp(-np.power(dist.numpy(), 2) / 400.)
        weight = torch.from_numpy(weight)
        input, target, weight = to_cuda_var(im), to_cuda_var(mask), to_cuda_var(weight)
        output = model(input)
        test_loss += loss_func(output, target, weight).data[0]
        label = threshold(-threshold(output, .5, 0, inplace=True), -.5, 1, inplace=True)
        if i == 0:
            n = 8
            comparison = torch.cat([input[:n], label[:n]])
            save_image(comparison.data.cpu(),
                     './results/test_' + str(epoch) + '.png', nrow=n)

    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

In [43]:
torch.cuda.empty_cache()
epochs = 20
for epoch in range(1, epochs + 1):
    train(epoch)
    test(epoch)
#


RuntimeError: a leaf Variable that requires grad has been used in an in-place operation.

In [13]:
torch.save(model.state_dict(), './models/segSVM_1')

In [39]:
a = torch.rand(1,2,3,4)
b = torch.rand(4)
c = a-b
c.size()

torch.Size([1, 2, 3, 4])